# **PiFuHD**

In [ ]:
!ngrok authtoken 2NY1EhEsrpOfzHL2d05uszWdmBz_2wzAHXkMx4iLvewqh97UX

In [ ]:
!ngrok config add-authtoken 2NY1EhEsrpOfzHL2d05uszWdmBz_2wzAHXkMx4iLvewqh97UX

In [ ]:
!pip install pyngrok flask_ngrok Flask-Cors

In [ ]:
pip install scikit-image==0.21.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 95.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.23.2
    Uninstalling scikit-image-0.23.2:
      Successfully uninstalled scikit-image-0.23.2


In [ ]:
!git clone https://github.com/facebookresearch/pifuhd
!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git

%cd /content/lightweight-human-pose-estimation.pytorch/
!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth

%cd /content/pifuhd/
!sh ./scripts/download_trained_model.sh

Cloning into 'pifuhd'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 222 (delta 92), reused 82 (delta 82), pack-reused 96
Receiving objects: 100% (222/222), 399.35 KiB | 1.54 MiB/s, done.
Resolving deltas: 100% (114/114), done.
Cloning into 'lightweight-human-pose-estimation.pytorch'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 124 (delta 21), reused 19 (delta 18), pack-reused 90
Receiving objects: 100% (124/124), 230.77 KiB | 1.21 MiB/s, done.
Resolving deltas: 100% (53/53), done.
/content/lightweight-human-pose-estimation.pytorch
--2024-07-30 21:19:20--  https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth
Resolving download.01.org (download.01.org)... 23.202.179.18, 2600:1406:bc00:f8a::a87, 2600:1406:bc00:f86::

In [ ]:
!pip install iopath portalocker
!pip install 'git+https://github.com/facebookresearch/pytorch3d.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31531 sha256=8e5c2c5f0f7364bff71e7695eef7323ac868cd8e02533355d1f2307e39e9f27b
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built iopath
  Cloning https://github.com/facebookresearch/pytorch3d.git to /tmp/pip-req-build-gjsnd5hj
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-gjsnd5hj
  Resolved https://github.com/facebookresearch/pytorch3d.git to commit 7edaee71a9583f8f1eedf91187aa77a0797c38e0
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch3d: filename=pytorch3d-0.7.7-cp310-cp310-linux_x86_64.whl size=57258553 sha256=e2ad05cb740e6df20a27dd97ea64e95f9cd133547deb5c6ec09983769250346a
  Stored in directory: /tmp/pip-ephem-wheel-cac

/content/lightweight-human-pose-estimation.pytorch


In [ ]:
%cd /content/lightweight-human-pose-estimation.pytorch/

import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo
from IPython.display import clear_output

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=int) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)

            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
                pmin = valid_keypoints.min(0)
                pmax = valid_keypoints.max(0)
                center = (0.5 * (pmax[:2] + pmin[:2])).astype(int)
                radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
                # if leg is missing, use pelvis to get cropping
                center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(int)
                radius = int(1.45 * np.sqrt(((center[None, :] - valid_keypoints) ** 2).sum(1)).max(0))
                center[1] += int(0.05 * radius)
            else:
                center = np.array([img.shape[1] // 2, img.shape[0] // 2])
                radius = max(img.shape[1] // 2, img.shape[0] // 2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2 * radius, 2 * radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

# Define the path to your image
image_path = 'D:/Ahmer/project/For backend Latest_002/uploads/best_frame_no_bg.png'  # Update this path to your actual image path

# Empty the folder
!rm -rf 'D:/Ahmer/project/For backend Latest_002/pifuhd/sample_images'
!mkdir 'D:/Ahmer/project/For backend Latest_002/pifuhd/sample_images'

import cv2
img = cv2.imread(image_path)
cv2.imwrite('D:/Ahmer/project/For backend Latest_002/pifuhd/sample_images/Img.png', img)

import os
image_path = 'D:/Ahmer/project/For backend Latest_002/pifuhd/sample_images/Img.png'
image_dir = os.path.dirname(image_path)
file_name = os.path.splitext(os.path.basename(image_path))[0]

# Output paths
obj_path = 'D:/Ahmer/project/For backend Latest_002/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name
out_img_path = 'D:/Ahmer/project/For backend Latest_002/pifuhd/results/pifuhd_final/recon/result_%s_256.png' % file_name
video_path = 'D:/Ahmer/project/For backend Latest_002/pifuhd/results/pifuhd_final/recon/result_%s_256.mp4' % file_name
video_display_path = 'D:/Ahmer/project/For backend Latest_002/pifuhd/results/pifuhd_final/result_%s_256_display.mp4' % file_name

# Preprocessing
%cd /content/lightweight-human-pose-estimation.pytorch/

net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)
get_rect(net.cuda(), [image_path], 512)

# Run
%cd /content/pifuhd/
# Warning: all images with the corresponding rectangle files under -i will be processed.
!python -m apps.simple_test -r 256 --use_rect -i $image_dir
# Seems that 256 is the maximum resolution that can fit into Google Colab.
# If you want to reconstruct a higher-resolution mesh, please try with your own machine.

# Clear everything
from IPython.display import clear_output
clear_output()

# Render video
from lib.colab_util import generate_video_from_obj, set_renderer, video

renderer = set_renderer()
generate_video_from_obj(obj_path, out_img_path, video_path, renderer)

# We cannot play a mp4 video generated by cv2
!ffmpeg -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet
video(video_display_path)



### **Taking Transparent Image To Convert**

In [ ]:
# Define the path to your image
image_path = '/content/best_frame_no_bg.png'  # Update this path to your actual image path

# Empty the folder
!rm -rf '/content/pifuhd/sample_images'
!mkdir '/content/pifuhd/sample_images'

import cv2
img = cv2.imread(image_path)
cv2.imwrite('/content/pifuhd/sample_images/Img.png', img)

import os
image_path = '/content/pifuhd/sample_images/Img.png'
image_dir = os.path.dirname(image_path)
file_name = os.path.splitext(os.path.basename(image_path))[0]

# Output paths
obj_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name
out_img_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.png' % file_name
video_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.mp4' % file_name
video_display_path = '/content/pifuhd/results/pifuhd_final/result_%s_256_display.mp4' % file_name

# Preprocessing
%cd /content/lightweight-human-pose-estimation.pytorch/

net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)
get_rect(net.cuda(), [image_path], 512)

# Run
%cd /content/pifuhd/
# Warning: all images with the corresponding rectangle files under -i will be processed.
!python -m apps.simple_test -r 256 --use_rect -i $image_dir
# Seems that 256 is the maximum resolution that can fit into Google Colab.
# If you want to reconstruct a higher-resolution mesh, please try with your own machine.

# Clear everything
from IPython.display import clear_output
clear_output()

# Render video
from lib.colab_util import generate_video_from_obj, set_renderer, video

renderer = set_renderer()
generate_video_from_obj(obj_path, out_img_path, video_path, renderer)

# We cannot play a mp4 video generated by cv2
!ffmpeg -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet
video(video_display_path)


In [ ]:
from google.colab import files
files.download('/content/pifuhd/results/pifuhd_final/recon/result_Img_256.obj')